In [1]:
import os
import random
import copy
import time
import numpy as np
import scipy.io

In [2]:
class nn(object):
    def __init__(self, layer):
        self.size = layer
        self.n = len(layer)
        self.hiddenLayer = self.n - 2
        self.activation_func = "sigm"
        self.learningRate = 0.01
        self.momentum = 0.95
        self.outputConnect = 1
        self.output = "softmax"
        
        # initiate weights and weight momentum for hidden layer
        self.W = {}
        self.vW = {}
        self.dW = {}
        self.c = {}
        #self.W = self.vW  = self.dW= self.c = {} 
        for i in range(1, self.n-1):
            self.W[i] = np.random.normal(0, np.sqrt(2/(self.size[i-1]+1)), size=(self.size[i],self.size[i - 1]+1))
            self.vW[i] = np.zeros(self.W[i].shape)
            self.dW[i] = np.zeros(self.W[i].shape)
            self.c[i] =  np.random.normal(0, np.sqrt(2/(self.size[i-1]+1)), size=(self.size[i - 1],1))
        
        #initiate weights and weight momentum for output layer
        self.Ws = {}
        self.vWs = {}
        self.dWs =  {}
        self.beta = {}
        self.betaOld = {}
        self.p= {}  
        if self.outputConnect == 1:
            for i in range(1,(self.hiddenLayer + 1)):
                self.Ws[i] = np.random.normal(0, np.sqrt(2/((self.W[i].shape[0])+1)), size=(self.size[len(self.size)-1],self.size[i]+1))
                self.vWs[i] = np.zeros(self.Ws[i].shape)
                self.dWs[i] = np.zeros(self.Ws[i].shape)         
                self.beta[i] = 1
                self.betaOld[i] = 1
                self.p[i]= 1 
        else:
            i = 1
            self.Ws[i] = np.random.normal(0, np.sqrt(2/((self.W[i-1].shape[0])+1)), size=(self.size[len(self.size)-1],self.size[len(self.size)-2]+1))
            self.vWs[i] = np.zeros(self.Ws[i].shape)
            self.dWs[i] = np.zeros(self.Ws[i].shape) 
            
        #initiate later used variable
        self.a = {}
        self.aas = {}
        self.e = {}
        self.L = {}
        self.classlabel = {}
        self.nop = {}
        self.nodes = {}

In [3]:
uhuy = nn([8,5,2])

In [4]:
uhuy.Ws[1].shape

(2, 6)

In [5]:

class Parameter(object):
    def __init__(self, nn,layer,K):
        self.nn = nn
        self.ev = {}
        self.size = layer
        self.prune_list       = 0;
        self.prune_list_index = [];
        self.ev[1] = {'layer': layer, 'kp':0, 'kl':0 ,'miu_x_old':0, 'var_x_old':0, 'kl':0,'K':K, 'cr':0,'node':{},
                     'BIAS2':{}, 'VAR':{}, 'miu_NS_old':0, 'var_NS_old':0, 'miu_NHS_old':0, 'var_NHS_old':0,
                     'miumin_NS':[], 'miumin_NHS':[], 'stdmin_NS':[], 'stdmin_NHS':[]}
        self.Loss = {}
        self.cr = {}
        self.wl = {}

In [6]:

class Performance(object):
    def __init__(self):
        self.update_time = 0
        self.ev = {}
        self.test_time = 0
        self.classification_time    = 0;
        self.layer = 0;
        self.ev[1] = {'f_measure': 0, 'g_mean':0, 'recall':0, 'precision':0}
        self.LayerWeight = 0
        self.meanode = []
        self.stdnode = []
        self.NumberOfParameters = 0

In [7]:

def sigmoid_array(x):                                        
    return 1 / (1 + np.exp(-x))

def relu_array(x):
    return abs(x) * (x > 0)

def softmax_array(x):
    list_re = []
    for arr in x:
        e_x = np.exp(arr - np.max(arr))
        result = e_x / e_x.sum()
        list_re.append(result)
    return np.array(list_re)  

def argmax_array(x):
    list_re = []
    for arr in x:
        result = [arr.argmax()]
        list_re.append(result)
    return np.array(list_re)  

def valuemax_array(x):
    list_re = []
    for arr in x:
        result = [arr.max()]
        list_re.append(result)
    return np.array(list_re)  

def update_beta(x):
    result = {}
    sum_val = sum(x.values())
    for k, v in x.items():
        result[k] = v/sum_val        
    return result

def checkbeta(dict1):
    count = 0
    for i in dict1.values():
        if(i != 0):
            count += 1
    return count

In [8]:
def testing(nn, x, y, ev):
    nop = []
    #feedforward
    nn = netfeedforward(nn,x,y)
    (row,m2) = y.shape
    decreasingFactor = 0.001
    #obtain true class
    act = argmax_array(y)
    for i in range(1,nn.hiddenLayer+1):
        nn.classlabel[i] = {}
    nn.sigma = np.zeros((row,m2))
    for t in range(0,row):
        for i in range(1,nn.hiddenLayer+1):
            if(nn.beta[i] != 0):
                #obtain predicted label, note : layer weight betaOld is fixed
                nn.sigma[t] = nn.sigma[t] + (nn.aas[i][t] * nn.betaOld[i])               
                nn.classlabel[i][t] = nn.aas[i][t].argmax()
                compare = act[t][0] - nn.classlabel[i][t]
                #train the weighted voting
                if(compare != 0):
                    nn.beta[i] = max(nn.beta[i] * nn.p[i], decreasingFactor)
                    nn.p[i] = max(nn.p[i] - decreasingFactor , decreasingFactor)
                elif(compare == 0):
                    nn.beta[i] = min(nn.beta[i] * (1+nn.p[i]), 1)
                    nn.p[i] = min(nn.p[i] + decreasingFactor , 1)
            #last element on data chunk        
            if(t==row):
                #calculate number of parameter
                if(nn.beta[i] != 0):
                    (c,d) = nn.Ws[i].shape
                    vw = 1
                else:
                    c = 0
                    d = 0
                    vw = 0
                (a,b) = nn.W[i].shape
                nop.append(a*b + c*d + vw) # number of parameters
                
                #calculate the number of node in each hidden layer
                nn.nodes[i][nn.t] = ev[1]['K']
        #print(nn.beta)
        nn.beta = update_beta(nn.beta);
    
    nn.nop[nn.t] = np.sum(nop)
    nn.mnop = [np.array(list(nn.nop.values())).mean(),np.array(list(nn.nop.values())).std()]
    
    #update voting weight
    nn.betaOld = nn.beta.copy()
    nn.index = max(nn.beta, key=nn.beta.get)
    
    
    #calculate classification rate
    [raw_out, out] = [valuemax_array(nn.sigma), argmax_array(nn.sigma)]
    nn.wrongClass = (np.where(out != act)[0]).reshape(-1,1)
    nn.cr = 1 - (nn.wrongClass.shape[0]/row)
    nn.residual_error = 1 - raw_out
    nn.actualLabel = out
    nn.act = act
    return [nn]
    #return nn,out,act,raw_out

In [9]:

def netfeedforward(nn,x,y):
    n = nn.n
    m = x.shape[0]
    ones = np.ones(shape= (m,1))
    x = np.append(ones, x, axis=1 ) #append columns of 1 in matrix for bias
    nn.a[1] = x
    #feedforward from input layer through all the hidden layer
    for i in range(2,n):
        if(nn.activation_func == 'sigm'):
            nn.a[i] = sigmoid_array(np.matmul(nn.a[i-1],nn.W[i-1].T))
        elif(nn.activation_func == 'relu'):
            nn.a[i] = relu_array(np.matmul(nn.a[i-1],nn.W[i-1].T))
        ones = np.ones(shape= (m,1))
        nn.a[i] = np.append(ones, nn.a[i], axis=1 )
    
    #propagate to the output layer
    for i in range(1,nn.hiddenLayer+1):
        if nn.beta[i] != 0:
            if(nn.output == 'sigm'):
                nn.aas[i] = sigmoid_array(np.matmul(nn.a[i+1], nn.Ws[i].T))
            elif(nn.output == 'linear'):
                nn.aas[i] = np.matmul(nn.a[i+1], nn.Ws[i].T)   
            elif(nn.output == 'softmax'):
                nn.aas[i] = softmax_array(np.matmul(nn.a[i+1],nn.Ws[i].T)) 
            
            
            #calculate error
            nn.e[i] = y - nn.aas[i]
               
            #calculate loss function
            if(nn.output == 'sigm' or nn.output == 'linear'):
                nn.L[i] = 0.5 * (np.sum(np.sum(nn.e[i]**2))) / m
            elif(nn.output == 'softmax'):
                nn.L[i] = -np.sum(np.sum(y * np.log(nn.aas[i]))) / m
        
            #print(nn.L) 
    return nn

In [10]:
#net configuration while training
def netconfigtrain(layer):
    net = nn(layer)
    net.layer = layer
    net.n = len(layer)
    net.activation_func = "sigm"
    net.learningRate = 0.01
    net.momentum = 0.95
    net.output = "softmax"
    return net

In [11]:

#calculate probit function
def probit(miu,std):
    p = (miu / (1+ np.pi * (std**2)/8) ** 0.5)
    return p

#calculate recursive mean and standard deviation
def meanstditer(miu_old, var_old,x,k):
    miu = miu_old + (x - miu_old) / k
    var = var_old + (x - miu_old) * (x-miu)
    std = np.sqrt(var/k)
  
    if(len(miu.shape) != 2):
        #print('len !=2')
        #print([np.array([miu]),np.array([std]),np.array([var])])
        return [np.array([miu]),np.array([std]),np.array([var])]
    else:
        return [miu,std,var]

In [13]:
def netFeedForwardWinner(nn,x,y):
    x = np.array([x])
    y = np.array([y])
    n = nn.n
    m = x.shape[0]
    nn.a[1] = x
   
    #feedforward form input layer through all the hidden layer 
    for i in range(2,n):
        if(nn.activation_func == 'sigm'):
            nn.a[i] = sigmoid_array(np.matmul(nn.a[i-1],nn.W[i-1].T))
        elif(nn.activation_func == 'relu'):
            nn.a[i] = relu_array(np.matmul(nn.a[i-1],nn.W[i-1].T))
        ones = np.ones(shape= (m,1))
        nn.a[i] = np.append(ones, nn.a[i], axis=1 )
  
    #propagate to the output layer
    if(nn.output == 'sigm'):
        nn.a[n] = sigmoid_array(np.matmul(nn.a[n-1],nn.W[n-1].T))
    elif(nn.output == 'linear'):
        nn.a[n] = np.matmul(nn.a[n-1],nn.W[n-1].T)   
    elif(nn.output == 'softmax'):
        nn.a[n] = softmax_array(np.matmul(nn.a[n-1], nn.W[n-1].T)) 
            
    #calculate error
    nn.e[1] = y - nn.a[n]
    return nn

In [14]:
#calculate backpropagation
def lossBackward(nn):
    n = nn.n
    d = {}
    if(nn.output == 'sigm'):
        d[n] = - nn.e[1] * (nn.a[n] * (1 - nn.a[n]))      
    elif(nn.output == 'linear' or nn.output == 'softmax'):
        d[n] = -1 * nn.e[1]
    #activation backward
    for i in range(n-1, 1, -1):
        if(nn.activation_func == 'sigm'):
            d_act = nn.a[i] * (1 - nn.a[i])
        elif(nn.activation_func == 'tanh_opt'):
            d_act = 1.7159 * 2/3 * (1 - 1 / (1.7159**2) * (nn.a[i]**2))   
        elif(nn.activation_func == 'relu'):
            d_act = np.zeros((1,len(nn.a[i])))
            for i in range(len(d_act)):
                print(type((nn.a[i + 1])))
                if(nn.a[i + 1].all()>0):
                    d_act[i] = 0             
        if(i+1 == n):
            d[i] = (np.matmul(d[i+1], nn.W[i]) * d_act)
        else:
            d[i] = (np.matmul(d[i+1][1:], nn.W[i]) * d_act)

    for i in range(1, n): 
        if(i+1 == n):
            nn.dW[i] = np.matmul(d[i+1].T, nn.a[i]) 
        else: 
            nn.dW[i] = np.matmul(d[i+1][:,1:].T, nn.a[i])   
    return nn

In [15]:

#update the weight
def optimizerStep(nn):
    for i in range(1, nn.n):
        dW = nn.dW[i]
        dW = nn.learningRate * dW       
        if(nn.momentum > 0):
            nn.vW[i] = nn.momentum * nn.vW[i] + dW
            dW = nn.vW[i]
        nn.W[i] = nn.W[i] - dW
    return nn

In [17]:
def training(parameter,y):
    bb = parameter.nn.W[parameter.nn.index].shape[1]
    grow = 0
    prune=0
    #initialize performance matrix
    ly          = parameter.nn.index
    kp          = parameter.ev[1]['kp']
    miu_x_old   = parameter.ev[1]['miu_x_old']
    var_x_old   = parameter.ev[1]['var_x_old']
    kl          = parameter.ev[ly]['kl']
    K           = parameter.ev[ly]['K']
    node        = parameter.ev[ly]['node']
    BIAS2       = parameter.ev[ly]['BIAS2']
    VAR         = parameter.ev[ly]['VAR']
    miu_NS_old  = parameter.ev[ly]['miu_NS_old']
    var_NS_old  = parameter.ev[ly]['var_NS_old']
    miu_NHS_old = parameter.ev[ly]['miu_NHS_old']
    var_NHS_old = parameter.ev[ly]['var_NHS_old']
    miumin_NS   = parameter.ev[ly]['miumin_NS']
    miumin_NHS  = parameter.ev[ly]['miumin_NHS']
    stdmin_NS   = parameter.ev[ly]['stdmin_NS']
    stdmin_NHS  = parameter.ev[ly]['stdmin_NHS']
    
    
    #initiate training model
    net = netconfigtrain([1,1,1])
    #substitute the weight to be trained to training model
    net.activation_func = parameter.nn.activation_func
    net.W[1]  = parameter.nn.W[ly]
    net.vW[1] = parameter.nn.vW[ly]
    net.dW[1] = parameter.nn.dW[ly]
    net.W[2]  = parameter.nn.Ws[ly]
    net.vW[2] = parameter.nn.vWs[ly]
    net.dW[2] = parameter.nn.dWs[ly]
    
    #load data in shuffled
    x = parameter.nn.a[ly]
    (N,I) = x.shape
    kk = np.random.permutation(N)
    x = x[kk]
    y = y[kk]
    
    #xavier initialization
    if(ly>1):
        n_in = parameter.ev[ly-1]['K']
    else:
        n_in = parameter.nn.size[0]

    miuNS       = np.zeros((N,1))
    miuminNS       = np.zeros((N,1))
    miuNHS       = np.zeros((N,1))
    miuminNHS       = np.zeros((N,1))

    
    #main loop, train the model
    for k in range(1, N+1):
        kp = kp+1
        kl = kl+1
        #incremental calculation of x_tail mean and variance
        [miu_x, std_x, var_x] = meanstditer(miu_x_old,var_x_old,parameter.nn.a[1][k-1],kp)
        miu_x_old = miu_x
        var_x_old = var_x

        #expectation of z
        py = probit(miu_x,std_x).T
      
        for ii in range(1, parameter.nn.index+1):
            if(ii == parameter.nn.index):
                py = sigmoid_array(np.matmul(net.W[1],py))
            else:
                py = sigmoid_array(np.matmul(parameter.nn.W[ii],py))
            py = np.append(np.array([[1]]),py, axis = 0)
            if(ii == 1):
                Ey2 = py**2

        Ey = py
        Ez = np.matmul(net.W[2], Ey) 
        Ez = softmax_array(Ez.T).T
        
        
        if(parameter.nn.hiddenLayer>1):
            py = Ey2
            for ii in range(2, parameter.nn.index+1):
                if(ii==parameter.nn.index):
                    py = sigmoid_array(np.matmul(net.W[1],py))
                else:
                    py = sigmoid_array(np.matmul(parameter.nn.W[ii],py))
                py = np.append(np.array([[1]]),py, axis = 0)
            Ey2 = py
        Ez2 = np.matmul(net.W[2], Ey2) 
        Ez2 = softmax_array(Ez2.T).T
          
            
        #Network mean calculation  
        bias2 = (Ez - np.array([[1,0]]).T)**2
        ns = bias2
        NS = np.linalg.norm(ns, 'fro')

        #Incremental calculation of NS mean and variance
        [miu_NS,std_NS,var_NS] = meanstditer(miu_NS_old,var_NS_old,NS,kp)
        miu_NS_old = miu_NS
        var_NS_old = var_NS
        miustd_NS  = miu_NS + std_NS
        miuNS[k-1] = miu_NS
        
        if(kl <= 1 or grow ==1):
            miumin_NS = miu_NS
            stdmin_NS = std_NS
        else:
            if(miu_NS < miumin_NS):
                miumin_NS = miu_NS
                stdmin_NS = std_NS
            if(std_NS < stdmin_NS):
                stdmin_NS = std_NS
        miuminNS[k-1] = miumin_NS
        miustdmin_NS  = miumin_NS + (1.3*np.exp(-NS)+0.7)*stdmin_NS
        BIAS2[kl] = miu_NS
        
        #growing hidden unit
        if(miustd_NS >= miustdmin_NS and kl>1):
            grow = 1
            K=K+1
            print('The new node no {} is FORMED around sample {}'.format(K, k))
            node[kp] = K
            W_app = np.random.normal(0, np.sqrt(2/(n_in+1)), size=(1, bb))
            net.W[1] = np.append(net.W[1],W_app, axis = 0)          
            net.vW[1] = np.append(net.vW[1],np.zeros((1,bb)), axis = 0)
            net.dW[1] = np.append(net.dW[1],np.zeros((1,bb)), axis = 0)
            
            W2_app = np.random.normal(0, np.sqrt(2/(K+1)), size=(parameter.nn.size[-1], 1))
            net.W[2] = np.append(net.W[2],W2_app, axis = 1)
            net.vW[2] = np.append(net.vW[2],np.zeros((parameter.nn.size[-1], 1)), axis = 1)
            net.dW[2] = np.append(net.dW[2],np.zeros((parameter.nn.size[-1], 1)), axis = 1)
            
            if(ly<parameter.nn.hiddenLayer):
                wNext = parameter.nn.W[ly+1].shape[0]
                parameter.nn.W[ly+1]  = np.append(parameter.nn.W[ly+1],np.random.normal(0, np.sqrt(2/(K+1)), size=(wNext, 1)), axis = 1)
                parameter.nn.vW[ly+1] = np.append(parameter.nn.vW[ly+1],np.zeros((wNext, 1)), axis = 1) 
                parameter.nn.dW[ly+1] = np.append(parameter.nn.dW[ly+1],np.zeros((wNext, 1)), axis = 1)         
        else:
            grow = 0
            node[kp] = K
        #Network variance calculation
#         ho = np.array([[0.7498],[0.2502]])
#         he = np.array([[0.7856],[0.2144]])
#         uy = ho-he**2
#         ey = np.linalg.norm(uy, 'fro')
       
        var = Ez2 - Ez**2
        NHS = np.linalg.norm(var, 'fro')
        
        # Incremental calculation of NHS mean and variance
        [miu_NHS,std_NHS,var_NHS] = meanstditer(miu_NHS_old,var_NHS_old,NHS,kp)
        miu_NHS_old = miu_NHS
        var_NHS_old = var_NHS
        miustd_NHS  = miu_NHS + std_NHS
        miuNHS[k-1] = miu_NHS
        
        if(kl <= I+1 or prune ==1):
            miumin_NHS = miu_NHS
            stdmin_NHS = std_NHS
        else:
            if(miu_NHS < miumin_NHS):
                miumin_NHS = miu_NHS
            if(std_NHS < stdmin_NHS):
                stdmin_NHS = std_NHS
        miuminNHS[k-1] = miumin_NHS
        miustdmin_NHS  = miumin_NHS + (2.6*np.exp(-NHS)+1.4)*stdmin_NHS
        VAR[kl] = miu_NHS
        
        # Pruning hidden unit
        if(grow == 0 and K>1 and miustd_NHS >= miustdmin_NHS and kl>I+1):
            HS = Ey[1:]
            BB = np.argmin(HS)
            print('The node no {} is PRUNED around sample {}'.format(BB+1, k))
            prune = 1
            K = K-1
            node[kp] = K
            net.W[1] = np.delete(net.W[1],BB,axis =0)
            net.vW[1] = np.delete(net.vW[1],BB,axis =0)
            net.dW[1] = np.delete(net.dW[1],BB,axis =0)
            
            net.W[2] = np.delete(net.W[2],BB+1,axis =1)
            net.vW[2] = np.delete(net.vW[2],BB+1,axis =1)
            net.dW[2] = np.delete(net.dW[2],BB+1,axis =1)
            if(ly<parameter.nn.hiddenLayer):
                parameter.nn.W[ly+1]  = np.delete(parameter.nn.W[ly+1],BB+1,axis =1)
                parameter.nn.vW[ly+1] = np.delete(parameter.nn.vW[ly+1],BB+1,axis =1)
                parameter.nn.dW[ly+1] = np.delete(parameter.nn.dW[ly+1],BB+1,axis =1)            
        else:
            prune = 0
            node[kp] = K
        #feedforward
        net = netFeedForwardWinner(net, x[k-1],y[k-1])
        
        #optimise parameter
        net = lossBackward(net)
        net = optimizerStep(net)
        
    #iterative learning
    ##unused
#     if nEpoch > 1
#     for iEpoch in range(1,nEpoch)
#         kk = randperm(nData)
#         x = x[kk,:]
#         y = y[kk,:]
#         for datum in range(nData)
#             %% feedforward
#             net = netFeedForwardWinner(net, x(iData,:), y(iData,:));
            
#             %% optimize the parameters
#             net = lossBackward(net);
#             net = optimizerStep(net);
#         end
    
    # subsitutte weight back to main model    
    parameter.nn.W[ly] = net.W[1]
    parameter.nn.Ws[ly] = net.W[2]
    
    #reset momentum and grandient
    parameter.nn.vW[ly] = net.vW[1] * 0
    parameter.nn.dW[ly] = net.dW[1] * 0
    parameter.nn.vWs[ly] = net.vW[2] * 0
    parameter.nn.dWs[ly] = net.dW[2] *0
    
    #substitute the recursive calculation
    parameter.ev[1]['kp']           = kp
    parameter.ev[1]['miu_x_old']    = miu_x_old
    parameter.ev[1]['var_x_old']    = var_x_old
    parameter.ev[ly]['kl']          = kl
    parameter.ev[ly]['K']           = K
    parameter.ev[ly]['node']        = node
    parameter.ev[ly]['BIAS2']       = BIAS2
    parameter.ev[ly]['VAR']         = VAR
    parameter.ev[ly]['miu_NS_old']  = miu_NS_old
    parameter.ev[ly]['var_NS_old']  = var_NS_old
    parameter.ev[ly]['miu_NHS_old'] = miu_NHS_old
    parameter.ev[ly]['var_NHS_old'] = var_NHS_old
    parameter.ev[ly]['miumin_NS']   = miumin_NS
    parameter.ev[ly]['miumin_NHS']  = miumin_NHS
    parameter.ev[ly]['stdmin_NS']   = stdmin_NS
    parameter.ev[ly]['stdmin_NHS']  = stdmin_NHS
    #print(BIAS2)             
    return parameter

In [18]:
#evaluation method
#method for precission, recall, f1 score and gmean
def performance_summary(act_raw, out_raw, nclass):
    Act = to_one_hot(act_raw, nclass)
    Out = to_one_hot(out_raw, nclass)
    recall = compute_recall(Act, Out, nclass)
    precission = compute_precission(Act, Out, nclass)
    f_measure = compute_f_measure(Act, Out, nclass)
    g_mean = compute_g_mean(recall, nclass)
    return [g_mean, f_measure, precission, recall]

def compute_g_mean(recall, nclass):
    g_mean = (np.prod(recall))**(1/nclass) 
    return g_mean

def compute_f_measure(Act, Out, nclass):
    f_measure = np.zeros((1,nclass))    
    for c in range(nclass):
        f_measure[0][c] = (2 * (Act[:,c].T.dot(Out[:,c]))) / (np.sum(Out[:,c]) + np.sum(Act[:,c]))
    f_measure[np.isnan(f_measure)] = 1
    return f_measure

def compute_precission(Act, Out, nclass):
    precission = np.zeros((1,nclass))    
    for c in range(nclass):
        precission[0][c] = (Act[:,c].T.dot(Out[:,c])) / np.sum(Out[:,c])
    precission[np.isnan(precission)] = 1
    return precission
    
def compute_recall(Act, Out, nclass):
    recall = np.zeros((1,nclass))  
    for c in range(nclass):
        recall[0][c] = (Act[:,c].T.dot(Out[:,c])) / np.sum(Act[:,c])
    recall[np.isnan(recall)] = 1
    return recall

def to_one_hot(x, nclass):
    y = np.zeros((x.shape[0],nclass))
    for i in range(len(x)):
        y[i][x[i]] = 1
    return y

In [19]:
#load data
data1 = scipy.io.loadmat('sea.mat')
data = data1.get('data')
n_feature = 3

In [20]:
# main method
def test_ADL(data, n_feature):   
    (nData, n_column) = data.shape
    M = n_column - n_feature
    preq_data = data[:,0:n_feature]
    preq_label = data[:,n_feature:]  
    chunk_size = 500
    no_of_chunk = int(nData/chunk_size)
    
    #dataProportion = 1 #unused variable
    
    drift = {}
    HL = {}
    buffer_x = []
    buffer_T = []
    tTest = []
    tTarget = []
    act = []
    out = []
    #initiate model 
    K = 1 #initial node
    network = nn([n_feature, K, M])
    
    #initiate node evolving iterative parameters
    layer = 1 #initial layer
    parameter = Parameter(network, layer,K)
    performance = Performance()
    
    # initiate drift detection parameter
    alpha_w = 0.0005;
    alpha_d = 0.0001;
    alpha   = 0.0001;
    
    #initiate layer merging iterative parameters
    covariance = np.zeros((5,5,M))
    covariance_old  = covariance
    threshold  = 0.05
    
    ClassificationRate = {}
    start_time = time.time()
    print('=========Autonomous Deep Learning has started=========\n')
    for count in range(0,no_of_chunk):  
        # prepare data
        n = count + 1
        minibatch_data  = preq_data [(n-1)*chunk_size:n*chunk_size]
        minibatch_label = preq_label[(n-1)*chunk_size:n*chunk_size]
        
        # neural network testing      
        print('Chunk: {} of {}'.format(n, no_of_chunk))
        print('Discriminative Testing: running ...')
        parameter.nn.t = n
        [parameter.nn] = testing(parameter.nn,minibatch_data,minibatch_label,parameter.ev)
        
        #metrics calculation
        parameter.Loss[n] = parameter.nn.L[parameter.nn.index]
        if(n == 1):
            tTest = parameter.nn.sigma.copy()
            act = parameter.nn.act.copy()
            out = parameter.nn.actualLabel.copy()
            parameter.residual_error = np.append(out,parameter.nn.residual_error,axis=0)
        else:
            tTest = np.append(tTest,parameter.nn.sigma,axis=0)
            act = np.append(act,parameter.nn.act,axis=0)
            out = np.append(out,parameter.nn.actualLabel,axis=0)
            parameter.residual_error = np.append(out,parameter.nn.residual_error,axis=0)
        parameter.cr[n] = parameter.nn.cr;
        ClassificationRate[n] = np.array(list(parameter.cr.values())).mean()
        print('Classification rate {}'.format(ClassificationRate[n]))
        print('Discriminative Testing: ... finished')
        
        #statistical measure
        performance.ev[n] = {}
        performance.NumberOfParameters = parameter.nn.mnop
        [performance.ev[n]['f_measure'], performance.ev[n]['g_mean'] ,performance.ev[n]['recall'],performance.ev[n]['precision']] = performance_summary(parameter.nn.act, parameter.nn.actualLabel, M)
        #last chunk only for testing process
        if(n == no_of_chunk):
            print('=========Parallel Autonomous Deep Learning is finished=========')
            performance.test_time = time.time() - start_time
            break
            
        #layer pruning mechanism
        outputCovar = np.zeros((layer,layer,M))
        for jj in range(1,layer+1):
            for kk in range(1, layer+1):
                if parameter.nn.beta[jj] != 0 and parameter.nn.beta[kk] != 0:
                    for ll in range(1,M+1):
                        temp = np.cov(parameter.nn.a[kk][:,ll],parameter.nn.a[jj][:,ll])
#                        print(temp[0][0])
                        outputCovar[jj-1,kk-1,ll-1] = temp[0][1]
                        covariance[jj-1,kk-1,ll-1] = ((covariance_old[jj-1,kk-1,ll-1]*(n)) + (n/n + 1)*outputCovar[jj-1,kk-1,ll-1] )/(n + 1)

        covariance_old = covariance
        if layer>1:
            merged_list = np.empty([2,2])
            for iter1 in range(layer -1):
                for hh in range(layer - iter1 -1):
                    if parameter.nn.beta[-2] != 0 or parameter.nn.beta[hh] != 0:
                        MCI = np.empty([2,2])
                        for o in range(M):
                            pearson = covariance[-1 - iter1,hh,o]/np.sqrt(covariance[-2,-2,o] * covariance[hh,hh,o])
                            MCI[o] = (0.5*(covariance[hh,hh,o] + covariance[-1 - iter1,-1 - iter1,o]) - np.sqrt((covariance[hh,hh,o] + covariance[-1 - iter1,-1 - iter1,o])**2 - 4*covariance[-1 - iter1,-1 - iter1,o]*covariance[hh,hh,o]*(1 - pearson**2)))
                        if max(abs(MCI)) < threshold:
                            if merged_list.size == 0:
                                merged_list[0,0] = layer -1
                                merged_list[0,1] = hh
                            else:
                                nu = np.where(merged_list[:,1:-2]==layer -1)
                                nu1 = np.where(merged_list[:,1:-2]== hh -1)
                                if np.array(nu).size == 0 and np.array(nu1).size == 0:
                                    n_list = np.array([layer-1,hh-1])
                                    merged_list = np.array(merged_list,nlist,axis=0)
                            break
            del_list = []
            for itt in range(merged_list.shape[0]):
                noOfHighlyCorrelated = np.where(merged_list[itt,:] == 0)
                if np.array(noOfHighlyCorrelated).size == 0 :
                    if parameter.net.beta[merged_list[itt,1]] > parameter.net.beta[merged_list[itt,2]]:
                        deleteLayer = merged_list[itt,2]
                    else:
                        deleteLayer = merged_list[itt,1]
                    del_list.append(deleteLayer)

            if len(del_list)>0 and parameter.nn.beta[del_list] != 0:
                print("'The Hidden Layer no {} is PRUNED around chunk {}]\n'".format(del_list, n))
                parameter.net.beta[del_list] = 0
            parameter.prune_list = parameter.prune_list + len[del_list]
            parameter.prune_list_index = np.append(parameter.prune_list_index,del_list)

        #Drift detection: output space
        if(n>1):
            cuttingpoint = 0
            pp = minibatch_label.shape[0]
            F_cut = np.zeros((pp,1))
            F_cut[parameter.nn.wrongClass] = 1
            Fupper = np.max(F_cut)
            Flower = np.min(F_cut)
            miu_F = np.mean(F_cut)
            
            for idx in range(pp):
                cut = idx + 1
                miu_G = np.mean(F_cut[0:cut])
                Gupper = np.max(F_cut[0:cut])
                Glower = np.min(F_cut[0:cut])
                epsilon_G = (Gupper - Glower) * np.sqrt(((pp)/(2*cut*(pp)) * np.log(1/alpha)))
                epsilon_F = (Fupper - Flower) * np.sqrt(((pp)/(2*cut*(pp)) * np.log(1/alpha)))
                if ((epsilon_G + miu_G) >= (miu_F + epsilon_F) and cut<pp):
                    cuttingpoint = cut
                    miu_H = np.mean(F_cut[(cuttingpoint):])
                    epsilon_D = (Fupper - Flower) * np.sqrt(((pp-cuttingpoint)/(2*cuttingpoint*(pp-cuttingpoint)) * np.log(1/alpha_d)))
                    epsilon_W = (Fupper - Flower) * np.sqrt(((pp-cuttingpoint)/(2*cuttingpoint*(pp-cuttingpoint)) * np.log(1/alpha_w)))
                    break
            if(cuttingpoint == 0):
                miu_H = miu_F
                epsilon_D = (Fupper - Flower) * np.sqrt(((pp)/(2*cut*(pp)) * np.log(1/alpha_d)))
                epsilon_W = (Fupper - Flower) * np.sqrt(((pp)/(2*cut*(pp)) * np.log(1/alpha_w)))
            
            #DRIFT STATUS
            if((np.abs(miu_G - miu_H)) > epsilon_D and cuttingpoint>1):
                st = 1
                print('Drift state: DRIFT')
                layer = layer+1
                parameter.nn.n = parameter.nn.n + 1
                parameter.nn.hiddenLayer = layer
                print('The new Layer no {} is FORMED around chunk {}'.format(layer, n))
                
                #Initiate NN weight parameters
                ii = parameter.nn.W[layer-1].shape[0]
                parameter.nn.W[layer] = np.random.normal(0,np.sqrt(2/(ii+1)),size = (1, (ii+1)))    
                parameter.nn.vW[layer] = np.zeros((1,ii+1))
                parameter.nn.dW[layer] = np.zeros((1,ii+1))
                
                #Initiate new classifier weight
                parameter.nn.Ws[layer]  = np.random.normal(0,1,size = (M,2))    
                parameter.nn.vWs[layer] = np.zeros((M,2))
                parameter.nn.dWs[layer] = np.zeros((M,2))
                
                #Initiate new voting weight
                parameter.nn.beta[layer] = 1
                parameter.nn.betaOld[layer] = 1
                parameter.nn.p[layer] = 1
                
                # Initiate iterative parameters
                parameter.ev[layer] = {}
                parameter.ev[layer]['layer ']      = layer
                parameter.ev[layer]['kl']          = 0
                parameter.ev[layer]['K']           = 1
                parameter.ev[layer]['cr']           = 0
                parameter.ev[layer]['node']        = {}
                parameter.ev[layer]['miu_NS_old']  = 0
                parameter.ev[layer]['var_NS_old']  = 0
                parameter.ev[layer]['miu_NHS_old'] = 0
                parameter.ev[layer]['var_NHS_old'] = 0
                parameter.ev[layer]['miumin_NS']   = []
                parameter.ev[layer]['miumin_NHS']  = []
                parameter.ev[layer]['stdmin_NS']   = []
                parameter.ev[layer]['stdmin_NHS']  = []
                parameter.ev[layer]['BIAS2']       = {}
                parameter.ev[layer]['VAR']         = {} 
                #initiate covariance
                covariance = np.zeros((5,5,M))
                covariance_old  = covariance
                
                #check buffer
                if(len(buffer_x) == 0):
                    pass
#                     h = parameter.nn.a[len(parameter.nn.a)][:,1:]
#                     z = minibatch_label
                else:
                    h = np.append(buffer_x[:,1:],parameter.nn.a[len(parameter.nn.a)][:,1:],axis=0)
                    if(len(buffer_T) == 0):
                        z = np.append(buffer_T,minibatch_label ,axis=0)
                    else:
                        z = minibatch_label
                        
                #Discriminative training for new layer
                    print('Discriminative Training for new layer: running ...')
                    parameter = netfeedforward(parameter,h,z)
                    print('Discriminative Training for new layer: ... finished')
                #clear buffer
                buffer_x = []
                buffer_T = []
            elif((np.abs(miu_G - miu_H)) >= epsilon_W and (np.abs(miu_G - miu_H)) < epsilon_D):
                st = 2
                print('Drift state: WARNING')
                buffer_x = minibatch_data
                buffer_T = minibatch_label
            else:
                st = 3
                print('Drift state: STABLE')
                if(len(buffer_x) == 0):
                    pass
                else:
#                    buffer_x = netffhl(parameter.nn, buffer_x)
                    h = np.append(buffer_x[:,1:],parameter.nn.a[len(parameter.nn.a)][:,1:],axis=0)
                    if(len(buffer_T) == 0):
                        z = np.append(buffer_T,minibatch_label ,axis=0)
                    else:
                        z = minibatch_label
                    print('Discriminative Training for new layer: running ...')
                    parameter = netfeedforward(parameter,h,z)
                    print('Discriminative Training for new layer: ... finished')
                buffer_x = []
                buffer_T = []
        else:
            st = 3
            print('Drift state: STABLE')
            buffer_x = []
            buffer_T = []
        drift[n] = st
        HL[n] = checkbeta(parameter.nn.beta)
        parameter.wl[n] = parameter.nn.index
        
        #Discriminative training for winning layer
        if(st != 1):
            print('Discriminative Training: running ...')
            parameter = training(parameter, minibatch_label)
            print('Discriminative Training: ... finished')
            
        #Clear current data chunk
        parameter.nn.a = {}
        print('=========Hidden layer number {} was updated========='.format(parameter.nn.index))
    return parameter,performance

In [21]:
if __name__ == "__main__":
    parameter, performance = test_ADL(data,n_feature)

=========Autonomous Deep Learning has started=========

Chunk: 1 of 200
Discriminative Testing: running ...
Classification rate 0.618
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
The new node no 2 is FORMED around sample 5
The node no 2 is PRUNED around sample 9
The new node no 2 is FORMED around sample 10
The new node no 3 is FORMED around sample 13
The node no 2 is PRUNED around sample 15
The new node no 3 is FORMED around sample 19
The new node no 4 is FORMED around sample 22
The new node no 5 is FORMED around sample 24
The new node no 6 is FORMED around sample 26
The node no 2 is PRUNED around sample 36
The new node no 6 is FORMED around sample 400
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 2 of 200
Discriminative Testing: running ...
Classification rate 0.6799999999999999
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
The node no 3 i

C:\Users\Bryan\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars



Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 3 of 200
Discriminative Testing: running ...
Classification rate 0.7506666666666666
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 4 of 200
Discriminative Testing: running ...
Classification rate 0.7755
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 5 of 200
Discriminative Testing: running ...
Classification rate 0.7979999999999999
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 6 of 200
Discriminative Testing: running ...
Classification rate 0.8056666666666666
Discr

Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 35 of 200
Discriminative Testing: running ...
Classification rate 0.8440571428571428
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 36 of 200
Discriminative Testing: running ...
Classification rate 0.8453333333333333
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 37 of 200
Discriminative Testing: running ...
Classification rate 0.8453513513513513
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 38 of 200
Discriminative Testing: running ...
Classification rate 0.8459999

Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 65 of 200
Discriminative Testing: running ...
Classification rate 0.8578461538461538
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 66 of 200
Discriminative Testing: running ...
Classification rate 0.8583333333333333
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 67 of 200
Discriminative Testing: running ...
Classification rate 0.8591940298507462
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 68 of 200
Discriminative Testing: running ...
Classification rate 0.8596470

Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 95 of 200
Discriminative Testing: running ...
Classification rate 0.8650315789473686
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 96 of 200
Discriminative Testing: running ...
Classification rate 0.8652916666666667
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 97 of 200
Discriminative Testing: running ...
Classification rate 0.8653402061855671
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 98 of 200
Discriminative Testing: running ...
Classification rate 0.8650204

Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 125 of 200
Discriminative Testing: running ...
Classification rate 0.8882080000000001
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 126 of 200
Discriminative Testing: running ...
Classification rate 0.8884603174603175
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 127 of 200
Discriminative Testing: running ...
Classification rate 0.8893070866141732
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 128 of 200
Discriminative Testing: running ...
Classification rate 0.889

Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 155 of 200
Discriminative Testing: running ...
Classification rate 0.9045290322580645
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 156 of 200
Discriminative Testing: running ...
Classification rate 0.905128205128205
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 157 of 200
Discriminative Testing: running ...
Classification rate 0.90571974522293
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 158 of 200
Discriminative Testing: running ...
Classification rate 0.906240

Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 185 of 200
Discriminative Testing: running ...
Classification rate 0.9170702702702703
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 186 of 200
Discriminative Testing: running ...
Classification rate 0.9174301075268817
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 187 of 200
Discriminative Testing: running ...
Classification rate 0.917850267379679
Discriminative Testing: ... finished
Drift state: STABLE
Discriminative Training: running ...
Discriminative Training: ... finished
=========Hidden layer number 1 was updated=========
Chunk: 188 of 200
Discriminative Testing: running ...
Classification rate 0.9182

In [22]:
parameter.nn.mnop

[0.0, 0.0]

In [23]:
[g,f,p,r] = performance_summary(act1[1:10],out1[1:10],2)

NameError: name 'act1' is not defined

In [ ]:
(g, f, p ,r)